# Let's Get That Data!

In [12]:
#First, The Imports
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [28]:
# the key variables
consumerKey = "XXXXXXXXX"
consumerSecret ="XXXXXXXXX"

In [29]:
#authentication
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)

#connecting to Twitter API
api = tweepy.API(auth)

In [34]:
#getting cat and dog search results
resultDog = api.search(q='%23Dog')
resultDoggo = api.search(q='%23doggo')
resultCat = api.search(q='%23Cat')

In [33]:
len(resultDog)

15

In [39]:
len(resultDoggo)

15

In [40]:
len(resultCat)

10